In [1]:
import torch
from torch.utils.data import DataLoader
from functools import partial
import transformers
from tqdm import tqdm
import pickle

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


In [3]:
import json

with open('val_qa.json') as f:
    d = json.load(f)
    print(len(d))

6415


In [5]:
d[1]

{'answer': 'man',
 'id': 30934,
 'question': 'who pours a seasoning liquid from a plastic container over chicken pieces placed in a plastic pouch?',
 'video_id': 1201}

In [3]:
from datasets import load_dataset

msvd_qa = load_dataset("morpheushoc/msvd-qa", split="train")


In [12]:

class MSVDQaDataset(torch.utils.data.Dataset):

    def __init__(self, module_path, dataset, task_type = 'qa', question_prompt = " Answer the question using a single word or a short phrase with multiple words."):
        
        self.data_list = []
        self.question_prompt = question_prompt

        self.movie_dir = module_path

        if (task_type == 'qa'):
            for elem in dataset:
                full_video_path = module_path + '/eval' +'/' + elem['id'].split("//")[1]
                for qa_pair in elem['qa']:
                    self.data_list.append({'full_video_path':full_video_path, 'data':qa_pair,'caption':elem['caption']})
                    
        elif (task_type == 'caption'):
            for elem in dataset:
                full_video_path = module_path + '/eval' +'/' + elem['video_path'].split("//")[1]
                self.data_list.append({'full_video_path':full_video_path, 'caption':elem['caption'],'data':None})

        else:
            print(f"Unsupported task type, return None")
            return None
            

        
    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        data_item = self.data_list[idx]
        video_path = data_item['full_video_path']
        
        if not os.path.exists(video_path):
            print(f"Warning: Video file not found at {video_path}, skipping this item.")
            return None  
    

        question, answer = data_item['data']
        captions = 'caption'
    
        return {
            'question': question + self.question_prompt,
            'video_path': video_path,
            #'pixel_values': pixel_values,
            'answer': answer,
            'captions': data_item['caption']
        }
    

In [13]:
dataset = MSVDQaDataset(module_path, msvd_qa, task_type = 'qa') 

## Inference

In [7]:
import os
import sys
import torch
import numpy as np
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [8]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_path, use_fast=False, padding_side='left')
tokenizer.pad_token = tokenizer.unk_token

In [14]:
def collate_fn(batches, tokenizer):
    
    questions = [_['question'] for _ in batches]
    video_path = [_['video_path'] for _ in batches]
    answer = [_['answer'] for _ in batches]
    captions = [_['captions'] for _ in batches]
    
    return questions, video_path, answer, captions

dataloader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=1,
        num_workers=1,
        pin_memory=True,
        drop_last=False,
        collate_fn=partial(collate_fn, tokenizer=tokenizer)
    )

iterator = iter(dataloader)
first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]
captions = first_batch[3]

print(question)
print(answer)
print(video_path)
print(captions)


first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]
captions = first_batch[3]

print(question)
print(answer)
print(video_path)
print(captions)

first_batch = next(iterator)


question = first_batch[0]
video_path = first_batch[1]
answer = first_batch[2]
captions = first_batch[3]

print(question)
print(answer)
print(video_path)
print(captions)

['what is chewing on a nut? Answer the question using a single word or a short phrase with multiple words.']
['animal']
['/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/YouTubeClips/-4wsuPCjDBc_5_15.avi']
[['a squirrel l is eating nuts', 'a squirrel is eating a nut', 'a hamster is eating a peanut', 'a chipmunk is eating a peanut', 'the squirrel ate the peanut out of the shell', 'an ardilla is eating', 'a rat is playing and eating something', 'the rabbit is eating', 'a squirrel is eating a whole peanut', 'a small animal is chewing on a nut', 'a ferret eats a leaf', 'a squirrel is eating a peanut in it s shell', 'a chipmunk is eating', 'a squirrel or some such animal eating a nut of some kind', 'a chipmunk is eating some food', 'a chipmunk is eating a nut', 'a squirrel is eating a peanut', 'a rabbit is eating', 'a ardilla coreana eating', 'a man is playing guitar', 'the squirrel is eating', 'a squirrel eating a fruit', 'a small squirrel is eating a peanut']]
['what is a s

In [ ]:
y_pred = []
y_real = []

progress_bar = tqdm(
        dataloader, total=len(dataloader), desc=f"Epoch 1"
    )

for step, batch in enumerate(progress_bar, start=1):
    print (step)
    question = batch[0]
    video_path = batch[1]
    answer = batch[2]
    captions = batch[3]

    # print(pixel_values.size())
    # print(question)
    # print(answer)
    # print(num_patches_list)
    # print(task_type)
    
    # with autocast():
        # 执行推理
    outputs = runner({"video": video_path[0], "text_input": question[0]}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]

    y_pred.append(outputs.strip())
    y_real.append(answer[0].strip())
    if (step%100 == 0):
        with open('y_pred_msvdqa.pkl', 'wb') as f:
            pickle.dump(y_pred, f)
        with open('y_real_msvdqa.pkl', 'wb') as f:
            pickle.dump(y_real, f)
        print("="*20 + "question" + "="*20)
        print (question[0])
        print("="*20 + "output" + "="*20)
        print(outputs)
        print("="*20 + "real answers" + "="*20)
        print(answer[0], flush = True)

Epoch 1:   0%|          | 0/30933 [00:00<?, ?it/s]

1


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be

2


Epoch 1:   0%|          | 2/30933 [00:28<103:18:00, 12.02s/it]

...

216


Epoch 1:   1%|          | 216/30933 [10:59<14:32:54,  1.71s/it]

217


Epoch 1:   1%|          | 216/30933 [11:02<26:10:55,  3.07s/it]


KeyboardInterrupt: 

In [21]:
y_pred[:10]

['According to the video, a nut is shown being chewed on by a rodent, which is a small rodent-like animal with a long whiskers.',
 "It's a small animal inside a cage.",
 'The animal eating a whole peanut is a small rodent that is seen eating a full peanut.',
 'The squirrel is eating a nut that is in a bowl, while standing on the table top in the cage. The video specifically shows that the squirrel is eating the nut and not the hay in the bowl, so there is no confusion about the nut in the video.',
 'A small squirrel is eating a peanut in a cage.',
 'A squirrel is eating some food from a dish in the video.',
 'The video shows a ferret sniffing the ground while eating out of a small bowl. There is no detailed information on what the ferret eats in the video.',
 'The chipmunk is eating food from a bowl, as shown in the video.',
 'The small squirrel in the video is eating from an empty Styrofoam cup, which is being held by a human. The cup is positioned on a small table and the squirrel is

In [22]:
y_real[:10]

['animal',
 'nut',
 'squirrel',
 'chipmunk',
 'chipmunk',
 'chipmunk',
 'leaf',
 'peanut',
 'peanut',
 'peanut']

## Evaluation by Gpt-3

In [ ]:
import openai

# Set your OpenAI API key
openai.api_key = "YOUR_API_KEY"

# Define the test cases
test_cases = [
    {"input": "What is the capital of France?", "expected_output": "Paris"},
    {"input": "What is 2 + 2?", "expected_output": "4"},
    {"input": "Explain photosynthesis briefly.", "expected_output": "Photosynthesis is the process by which plants convert sunlight into energy."},
]

# Validate the model
for test in test_cases:
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": test["input"]}]
    )
    output = response['choices'][0]['message']['content'].strip()
    print(f"Input: {test['input']}")
    print(f"Expected: {test['expected_output']}")
    print(f"Output: {output}")
    print(f"Match: {output == test['expected_output']}")
    print("-" * 30)